# Proof of concept: *PyGMQL*

In [3]:
import gmql as gl

## Loading a dataset 

In [6]:
# path of the dataset
input_path = "/home/luca/Scrivania/GMQL-Python/resources/hg_narrowPeaks/"

The library provides the user with a set of different parsers for datasets. In particular, for this demonstration we use the `NarrowPeakParser`.

In [7]:
np_parser = gl.parsers.NarrowPeakParser()

One of the main abstractions of the library is the `GMQLDataset`. It is the main access point of the user to the data. Each data manipulation operation is operated on a `GMQLDataset` and returns a *new* object of the same type.

In [8]:
dataset = gl.GMQLDataset(parser=np_parser)

In [9]:
dataset = dataset.load_from_path(path=input_path)

2017-03-28 12:02:46,744 - gmql_logger - INFO - loading metadata
2017-03-28 12:02:49,517 - gmql_logger - INFO - parsing metadata
2017-03-28 12:02:49,518 - gmql_logger - INFO - collecting metadata
2017-03-28 12:02:57,790 - gmql_logger - INFO - dataframe construction


100%|██████████| 115/115 [00:39<00:00,  2.96it/s]

2017-03-28 12:03:37,578 - gmql_logger - INFO - loading region data


2017-03-28 12:03:38,302 - gmql_logger - INFO - parsing region data


## Metadata management

Differently from the GMQL query engine, PyGMQL stores metadata directly in local memory as a **pandas dataframe** whose index (`id_sample`) is the sample id generated by the GMQL engine (based on the hash of the file name from which the sample comes from).
Each column of the dataframe represents one of the found attributes in the dataset, therefore it is possible that a sample (a row) has zero values for one column.

Other important fact is that each cell of the metadata dataframe is a **list** due to the fact that an attribute can have multiple values.

In [10]:
# for visualization purposes we only show 3 columns of 
# the dataframe (for a total of 115) and we use the 'head' 
# function to show only the first rows of the dataframe
dataset.meta_dataset.head()[['antibody','cell','antibody_lab']]

,antibody,cell,antibody_lab
id_sample,,,
-9220584259719780958,[E2F6],[HeLa-S3],"[Farnham, Myers]"
-9219803729611809025,[CTCF],[AG04449],"[Myers, Hardison, Snyder]"
-9211745969161520790,[eGFP-HDAC8],[K562],[White]
-9210823539931286134,[GABP],[H1-hESC],[Myers]
-9197969371849812436,[],[HPF],[]


### Select based on metadata with a logical predicate

We select the samples of the dataset in which 'antibody' has 'CTCF' value. 
The function to be applied is the `meta_select` which accepts a generic predicate, which is basically an arbitrary complex function; in this example we use a lambda expression.
The selection affects both metadata and region samples.

In [11]:
filtered_dataset = dataset.meta_select(lambda row: 'CTCF' in row['antibody'])

In [12]:
# visualize only the first rows of the metadata pandas dataframe
filtered_dataset.meta_dataset.head()[['antibody','cell','antibody_lab']]

,antibody,cell,antibody_lab
id_sample,,,
-9219803729611809025,[CTCF],[AG04449],"[Myers, Hardison, Snyder]"
-9120762041249846625,[CTCF],[MCF-7],"[Myers, Hardison, Snyder]"
-9118037537398139811,[CTCF],[MCF-7],"[Myers, Hardison, Snyder]"
-8760850962206896694,[CTCF],[MCF-7],"[Myers, Hardison, Snyder]"
-8556045950597285261,[CTCF],[A549],"[Myers, Hardison, Snyder]"


We can use the function `get_reg_sample(n)` to materialize a little sample of the regions in memory to a pandas dataframe

In [13]:
filtered_dataset.get_reg_sample(1)

,chr,id_sample,name,pValue,peak,qValue,score,signalValue,start,stop,strand
0,chr12,-4554121533430493468,.,-1.0,203.0,4.847566,1000.0,526.599159,58299107,58299514,*


### Project metadata based on an attribute list

An other possible metadata operation is the `meta_project` which, in its simplest form, takes only the specified columns of the dataframe and (as always) returns a new `GMQLDataset`

In [ ]:
filtered_proj_data = filtered_dataset.meta_project(['antibody', 'cell'])
filtered_proj_data.meta_dataset.head()

### Add a new column

If the user wants to add a new attribute to the metadata (basically a new column of the dataframe) he needs to call the `add_meta` function that takes the name of the new attribute and the default value to assign to each sample of the dataset

In [ ]:
filtered_proj_data = filtered_proj_data.add_meta('creator', 'luca')
filtered_proj_data.meta_dataset.head()

In [ ]:
# we can visualize all the attribute names
all_attributes = filtered_proj_data.get_meta_attributes()
all_attributes

### Project and also compute new columns based on complex functions

The `meta_project` function can take an other argument which is a dictionary of the following type:
```
    new_attributes = {
        'new_attribute_name_1' : complex_function_1,
        'new_attribute_name_2' : complex_function_2,
        ...
        
        'new_attribute_name_N' : complex_function_N,
    }
```
This argument enables the user to build new columns/attributes of the metadata dataframe based on the values of the other attributes.

In [ ]:
# define a function that operates on rows of the metadata dataset and 
# gives us the resulting new column value

# in particular this function simply concatenates the lists of 
# antibody and cell values
def complex_function(row):
    x = list(row['antibody'])
    y = list(row['cell'])
    #print("antibody: {}\t cell: {}".format(x, y))
    return x + y

In [ ]:
new_attr_dict = {
    'extended' : complex_function
}

extended_dataset = filtered_proj_data.meta_project(attr_list=all_attributes,
                                                   new_attr_dict=new_attr_dict)

In [ ]:
extended_dataset.meta_dataset.head()

## _Example: working with metadata_

We demonstrate the usage of some of the function described above with a very simple (and stupid) example.
The user adds two new attributes (the same for every sample) describing birth date and death date of the patient.
Then he generates a third new attribute given by the other two that represents the age of the patient.

In [ ]:
from datetime import datetime

born_date = datetime.strptime("30 Nov 1935","%d %b %Y")
death_date = datetime.strptime("30 Nov 1999","%d %b %Y")

In [ ]:
example_dataset = filtered_proj_data.add_meta('born_date', born_date)
example_dataset = example_dataset.add_meta('death_date', death_date)
all_attributes = example_dataset.get_meta_attributes()
all_attributes

In [ ]:
def calculate_age(row):
    #print(row)
    born_date = row['born_date'][0]
    death_date = row['death_date'][0]
    return (death_date - born_date).days / 365

In [ ]:
new_attr_dict = {
    'age' : calculate_age
}
example_dataset = example_dataset.meta_project(attr_list=all_attributes,
                                               new_attr_dict=new_attr_dict)

In [ ]:
example_dataset.meta_dataset.head()

## Region management

### Region selection

In [ ]:
ctcf_mcf7_dataset = dataset.meta_select(lambda attr: 'CTCF' in attr['antibody']
                                        and 'MCF-7' in attr['cell'])\
                           .reg_select(lambda reg: reg['pValue'] < 2)

In [ ]:
ctcf_mcf7_dataset.get_reg_sample(20)